# Calib sensor functions

NB study: module calib_sensor_functions.py in invisible_cities/reco

In [1]:
import os
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

import time
import glob
import tables as tb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from functools import reduce
%matplotlib inline
%load_ext autoreload
%autoreload 2

plt.rcParams["figure.figsize"] = 10, 8
plt.rcParams["font.size"     ] = 14

2017-12-03 11:13:36


In [2]:
from collections import namedtuple

In [3]:
from invisible_cities.reco import calib_sensors_functions_c as cpf

In [4]:
NSIPM = 4
WL    = 10
adc_to_pes = 10
adc_to_pes  = np.full(NSIPM, adc_to_pes, dtype=np.double)

In [5]:
adc_to_pes

array([ 10.,  10.,  10.,  10.])

In [6]:
signal_adc  = np.random.randint(1000, 1100, size=(NSIPM, WL), dtype=np.int16)

In [7]:
signal_adc

array([[1021, 1061, 1046, 1012, 1016, 1007, 1077, 1093, 1071, 1034],
       [1061, 1073, 1077, 1042, 1037, 1069, 1094, 1013, 1015, 1079],
       [1009, 1086, 1094, 1003, 1046, 1062, 1069, 1032, 1065, 1055],
       [1084, 1090, 1071, 1021, 1072, 1056, 1027, 1018, 1043, 1092]], dtype=int16)

In [8]:
signal_pes  = signal_adc - np.mean(signal_adc, axis=1)[:, np.newaxis]

In [9]:
signal_pes

array([[-22.8,  17.2,   2.2, -31.8, -27.8, -36.8,  33.2,  49.2,  27.2,
         -9.8],
       [  5. ,  17. ,  21. , -14. , -19. ,  13. ,  38. , -43. , -41. ,  23. ],
       [-43.1,  33.9,  41.9, -49.1,  -6.1,   9.9,  16.9, -20.1,  12.9,
          2.9],
       [ 26.6,  32.6,  13.6, -36.4,  14.6,  -1.4, -30.4, -39.4, -14.4,
         34.6]])

In [10]:
signal_pes /= adc_to_pes[:, np.newaxis]

In [11]:
signal_pes

array([[-2.28,  1.72,  0.22, -3.18, -2.78, -3.68,  3.32,  4.92,  2.72,
        -0.98],
       [ 0.5 ,  1.7 ,  2.1 , -1.4 , -1.9 ,  1.3 ,  3.8 , -4.3 , -4.1 ,  2.3 ],
       [-4.31,  3.39,  4.19, -4.91, -0.61,  0.99,  1.69, -2.01,  1.29,
         0.29],
       [ 2.66,  3.26,  1.36, -3.64,  1.46, -0.14, -3.04, -3.94, -1.44,
         3.46]])

In [12]:
signal_zs_common_threshold = np.array(signal_pes)

In [13]:
signal_zs_common_threshold 

array([[-2.28,  1.72,  0.22, -3.18, -2.78, -3.68,  3.32,  4.92,  2.72,
        -0.98],
       [ 0.5 ,  1.7 ,  2.1 , -1.4 , -1.9 ,  1.3 ,  3.8 , -4.3 , -4.1 ,  2.3 ],
       [-4.31,  3.39,  4.19, -4.91, -0.61,  0.99,  1.69, -2.01,  1.29,
         0.29],
       [ 2.66,  3.26,  1.36, -3.64,  1.46, -0.14, -3.04, -3.94, -1.44,
         3.46]])

In [14]:
common_threshold      = np.random.uniform(0.3, 0.7)

In [15]:
common_threshold 

0.5055501657635819

In [16]:
signal_zs_common_threshold[signal_pes < common_threshold] = 0

In [17]:
signal_zs_common_threshold

array([[ 0.  ,  1.72,  0.  ,  0.  ,  0.  ,  0.  ,  3.32,  4.92,  2.72,  0.  ],
       [ 0.  ,  1.7 ,  2.1 ,  0.  ,  0.  ,  1.3 ,  3.8 ,  0.  ,  0.  ,  2.3 ],
       [ 0.  ,  3.39,  4.19,  0.  ,  0.  ,  0.99,  1.69,  0.  ,  1.29,  0.  ],
       [ 2.66,  3.26,  1.36,  0.  ,  1.46,  0.  ,  0.  ,  0.  ,  0.  ,
         3.46]])

In [18]:
 individual_thresholds = np.random.uniform(0.3, 0.7, size=NSIPM)

In [19]:
individual_thresholds

array([ 0.4422847 ,  0.32636096,  0.51381197,  0.57255859])

In [20]:
individual_thresholds_reshaped = individual_thresholds[:, np.newaxis]

In [21]:
individual_thresholds_reshaped

array([[ 0.4422847 ],
       [ 0.32636096],
       [ 0.51381197],
       [ 0.57255859]])

In [22]:
signal_zs_individual_thresholds = np.array(signal_pes)
signal_zs_individual_thresholds[signal_pes < individual_thresholds_reshaped] = 0

In [23]:
signal_zs_individual_thresholds

array([[ 0.  ,  1.72,  0.  ,  0.  ,  0.  ,  0.  ,  3.32,  4.92,  2.72,  0.  ],
       [ 0.5 ,  1.7 ,  2.1 ,  0.  ,  0.  ,  1.3 ,  3.8 ,  0.  ,  0.  ,  2.3 ],
       [ 0.  ,  3.39,  4.19,  0.  ,  0.  ,  0.99,  1.69,  0.  ,  1.29,  0.  ],
       [ 2.66,  3.26,  1.36,  0.  ,  1.46,  0.  ,  0.  ,  0.  ,  0.  ,
         3.46]])

### Generate signal

In [24]:
def toy_sipm_signal(nsipm=4, wl=10, adc_to_pes=10, signal_range=(1000, 1100), threshold_range=(0.3, 0.7)):
    
    common_threshold      = np.random.uniform(threshold_range[0], threshold_range[1])
    individual_thresholds = np.random.uniform(threshold_range[0], threshold_range[1], size=nsipm)

    adc_to_pes  = np.full(nsipm, adc_to_pes, dtype=np.double)
    signal_adc  = np.random.randint(signal_range[0], signal_range[1], size=(nsipm, wl), dtype=np.int16)

    # subtract baseline and convert to pes
    signal_pes  = signal_adc - np.mean(signal_adc, axis=1)[:, np.newaxis]
    signal_pes /= adc_to_pes[:, np.newaxis]

    signal_zs_common_threshold = np.array(signal_pes)
    signal_zs_common_threshold[signal_pes < common_threshold] = 0

    # thresholds must be reshaped to allow broadcasting
    individual_thresholds_reshaped = individual_thresholds[:, np.newaxis]

    signal_zs_individual_thresholds = np.array(signal_pes)
    signal_zs_individual_thresholds[signal_pes < individual_thresholds_reshaped] = 0

    return (namedtuple('SipmSignal',
                        """signal_adc adc_to_pes signal_zs_common_threshold
                           signal_zs_individual_thresholds common_threshold
                           individual_thresholds""")
        (
         signal_adc                      = signal_adc, 
         adc_to_pes                      = adc_to_pes,
         signal_zs_common_threshold      = signal_zs_common_threshold,
         signal_zs_individual_thresholds = signal_zs_individual_thresholds,
         common_threshold                = common_threshold,
         individual_thresholds           = individual_thresholds))

In [25]:
ss = toy_sipm_signal()

In [26]:
print("""signal_adc = {}

         adc_to_pes = {}     
         
         signal_zs_common_threshold      = {}
         
         signal_zs_individual_thresholds = {}
         
         common_threshold                = {}
         
         individual_thresholds           = {}
      """.format(ss.signal_adc, ss.adc_to_pes, 
                 ss.signal_zs_common_threshold, ss.signal_zs_individual_thresholds,
                 ss.common_threshold, ss.individual_thresholds))

signal_adc = [[1002 1053 1014 1081 1081 1041 1051 1075 1026 1075]
 [1056 1014 1002 1059 1003 1029 1097 1051 1006 1075]
 [1066 1065 1095 1060 1021 1083 1059 1082 1093 1003]
 [1045 1023 1053 1016 1049 1061 1056 1069 1077 1014]]

         adc_to_pes = [ 10.  10.  10.  10.]     
         
         signal_zs_common_threshold      = [[ 0.    0.    0.    3.11  3.11  0.    0.    2.51  0.    2.51]
 [ 1.68  0.    0.    1.98  0.    0.    5.78  1.18  0.    3.58]
 [ 0.    0.    3.23  0.    0.    2.03  0.    1.93  3.03  0.  ]
 [ 0.    0.    0.67  0.    0.    1.47  0.97  2.27  3.07  0.  ]]
         
         signal_zs_individual_thresholds = [[ 0.    0.    0.    3.11  3.11  0.    0.    2.51  0.    2.51]
 [ 1.68  0.    0.    1.98  0.    0.    5.78  1.18  0.    3.58]
 [ 0.    0.    3.23  0.    0.    2.03  0.    1.93  3.03  0.  ]
 [ 0.    0.    0.67  0.    0.    1.47  0.97  2.27  3.07  0.  ]]
         
         common_threshold                = 0.5675704857939541
         
         individual_thresholds

### Verify that old function yields the same results than new functions

In [29]:
ss = toy_sipm_signal()
zs_wf0 = cpf._signal_sipm(ss.signal_adc, ss.adc_to_pes, ss.common_threshold, n_MAU=100, Cal=0)
zs_wf1 = cpf._signal_sipm(ss.signal_adc, ss.adc_to_pes, ss.common_threshold, n_MAU=100, Cal=1)
zs_wf2 = cpf._signal_sipm(ss.signal_adc, ss.adc_to_pes, ss.common_threshold, n_MAU=100, Cal=2)
zs_xf0 = cpf.sipm_signal_above_thr_mau(ss.signal_adc, ss.adc_to_pes, ss.common_threshold, n_MAU=100)
zs_xf1 = cpf.sipm_subtract_baseline_and_normalize_mau(ss.signal_adc, ss.adc_to_pes, n_MAU=100)
zs_xf2 = cpf.sipm_subtract_baseline_and_normalize(ss.signal_adc, ss.adc_to_pes)

In [31]:
print('zs_wf0 = {}\n  zs_xf0= {}'.format(zs_wf0, zs_xf0))

zs_wf0 = [[ 0.    0.    0.    3.92  0.72  2.52  1.52  3.42  0.    0.  ]
 [ 0.    3.48  1.08  0.    2.28  0.    4.38  0.48  0.    0.  ]
 [ 0.    0.    2.79  0.39  0.89  0.    0.    3.39  3.09  0.  ]
 [ 0.    1.96  1.56  3.86  0.    0.    2.26  0.    0.    1.36]]
  zs_xf0= [[ 0.    0.    0.    3.92  0.72  2.52  1.52  3.42  0.    0.  ]
 [ 0.    3.48  1.08  0.    2.28  0.    4.38  0.48  0.    0.  ]
 [ 0.    0.    2.79  0.39  0.89  0.    0.    3.39  3.09  0.  ]
 [ 0.    1.96  1.56  3.86  0.    0.    2.26  0.    0.    1.36]]


In [32]:
print('zs_wf1 = {}\n  zs_xf1= {}'.format(zs_wf1, zs_xf1))

zs_wf1 = [[-4.58 -2.88 -1.58  3.92  0.72  2.52  1.52  3.42 -0.28 -2.78]
 [-1.62  3.48  1.08 -3.42  2.28 -1.92  4.38  0.48 -0.12 -4.62]
 [-0.21 -0.71  2.79  0.39  0.89 -3.41 -3.31  3.39  3.09 -2.91]
 [-2.04  1.96  1.56  3.86 -2.84 -2.04  2.26 -3.44 -0.64  1.36]]
  zs_xf1= [[-4.58 -2.88 -1.58  3.92  0.72  2.52  1.52  3.42 -0.28 -2.78]
 [-1.62  3.48  1.08 -3.42  2.28 -1.92  4.38  0.48 -0.12 -4.62]
 [-0.21 -0.71  2.79  0.39  0.89 -3.41 -3.31  3.39  3.09 -2.91]
 [-2.04  1.96  1.56  3.86 -2.84 -2.04  2.26 -3.44 -0.64  1.36]]


In [33]:
print('zs_wf2 = {}\n  zs_xf2= {}'.format(zs_wf2, zs_xf2))

zs_wf2 = [[-4.58 -2.88 -1.58  3.92  0.72  2.52  1.52  3.42 -0.28 -2.78]
 [-1.62  3.48  1.08 -3.42  2.28 -1.92  4.38  0.48 -0.12 -4.62]
 [-0.21 -0.71  2.79  0.39  0.89 -3.41 -3.31  3.39  3.09 -2.91]
 [-2.04  1.96  1.56  3.86 -2.84 -2.04  2.26 -3.44 -0.64  1.36]]
  zs_xf2= [[-4.58 -2.88 -1.58  3.92  0.72  2.52  1.52  3.42 -0.28 -2.78]
 [-1.62  3.48  1.08 -3.42  2.28 -1.92  4.38  0.48 -0.12 -4.62]
 [-0.21 -0.71  2.79  0.39  0.89 -3.41 -3.31  3.39  3.09 -2.91]
 [-2.04  1.96  1.56  3.86 -2.84 -2.04  2.26 -3.44 -0.64  1.36]]


In [35]:
np.testing.assert_allclose(zs_wf0, zs_xf0, atol=1e-10)
np.testing.assert_allclose(zs_wf1, zs_xf1, atol=1e-10)
np.testing.assert_allclose(zs_wf2, zs_xf2, atol=1e-10)

### Verify that pedestal subtracted near baseline

In [36]:
def gaussian_sipm_signal(nsipm = 40, wfl = 100):
    
    sipm = np.zeros(nsipm * wfl, dtype=np.int16)
    sipm = np.reshape(sipm,(nsipm,wfl))
    for i in range(nsipm):
        sipm[i,:] = np.random.normal(1000+i*10, 1, wfl)
    
    NSiPM = sipm.shape[0]
    NSiWF = sipm.shape[1]
    print('NSiPM = {}, NSiWF = {}'.format(NSiPM, NSiWF))

    adc_to_pes = np.abs(np.random.normal(1, 0.01, nsipm))
    #print('adc_to_pes = {}'.format(adc_to_pes))
    #print('sipm array = {}'.format(sipm))
    #print('baselines = {}'.format(np.mean(sipm,axis=1)))
    return (sipm, adc_to_pes)

In [37]:
gaussian_sipm_signal(nsipm = 4, wfl = 10)

NSiPM = 4, NSiWF = 10


(array([[ 998,  999,  998,  999, 1000, 1000, 1001, 1001,  999,  999],
        [1008, 1010, 1009, 1009, 1009, 1010, 1008, 1011, 1010, 1009],
        [1019, 1019, 1020, 1019, 1020, 1020, 1018, 1019, 1020, 1019],
        [1031, 1029, 1028, 1029, 1030, 1033, 1030, 1030, 1030, 1030]], dtype=int16),
 array([ 1.01784749,  1.00200415,  0.99389925,  0.9905818 ]))

In [38]:
sipm, adc_to_pes = gaussian_sipm_signal(nsipm = 4, wfl = 100)

NSiPM = 4, NSiWF = 100


In [39]:
np.mean(sipm, axis=1)

array([  999.3 ,  1009.5 ,  1019.42,  1029.22])

In [40]:
wf = cpf.sipm_subtract_baseline_and_normalize(sipm, adc_to_pes)

In [41]:
np.mean(wf, axis=1)

array([  4.51239046e-14,   1.33226763e-17,   4.12792023e-14,
        -2.72892819e-14])

In [42]:
wf2 = cpf.sipm_subtract_baseline_and_normalize_mau(sipm, adc_to_pes, n_MAU=10)

In [43]:
np.mean(wf2, axis=1)

array([  4.51239046e-14,   1.33226763e-17,   4.12792023e-14,
        -2.72892819e-14])

In [44]:
np.testing.assert_allclose(np.mean(wf2, axis=1), 0, atol=1e-10)

In [45]:
np.testing.assert_allclose(np.mean(wf, axis=1), 0, atol=1e-10)

### Tests

In [48]:
def test_signal_sipm_common_threshold():
    ss = toy_sipm_signal()

    zs_wf = cpf._signal_sipm(ss.signal_adc, ss.adc_to_pes, ss.common_threshold)
    assert np.allclose(zs_wf, ss.signal_zs_common_threshold)


In [49]:
test_signal_sipm_common_threshold()

In [52]:
def test_signal_sipm_common_threshold_new():
    ss = toy_sipm_signal()

    zs_wf = cpf.sipm_signal_above_thr_mau(ss.signal_adc, ss.adc_to_pes, ss.common_threshold, n_MAU=100)
    assert np.allclose(zs_wf, ss.signal_zs_common_threshold)



In [53]:
test_signal_sipm_common_threshold_new()

In [54]:
ss = toy_sipm_signal(nsipm=1792, wl=1000, adc_to_pes=10, signal_range=(1000, 1100), threshold_range=(0.3, 0.7))

In [55]:
test_signal_sipm_common_threshold_new()

### Performance

In [56]:
ss = toy_sipm_signal(nsipm=1792, wl=1000, adc_to_pes=10, signal_range=(1000, 1100), threshold_range=(0.3, 0.7))

In [57]:
%timeit cpf._signal_sipm(ss.signal_adc, ss.adc_to_pes, ss.common_threshold)

191 ms ± 10.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [58]:
%timeit cpf._signal_sipm(ss.signal_adc, ss.adc_to_pes, ss.individual_thresholds)

194 ms ± 13.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [59]:
%timeit cpf.sipm_signal_above_thr_mau(ss.signal_adc, ss.adc_to_pes, ss.common_threshold, n_MAU=100)

220 ms ± 4.79 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [60]:
%timeit cpf.sipm_signal_above_thr_mau(ss.signal_adc, ss.adc_to_pes, ss.individual_thresholds, n_MAU=100)

221 ms ± 4.22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [61]:
%timeit cpf.sipm_subtract_baseline_and_normalize_mau(ss.signal_adc, ss.adc_to_pes)

188 ms ± 13.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [62]:
%timeit cpf.sipm_subtract_baseline_and_normalize(ss.signal_adc, ss.adc_to_pes)

13.6 ms ± 356 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### NB: simple pedestal subtraction is 10 times fsster than MAU functions

In [63]:
%timeit cpf._signal_sipm(ss.signal_adc, ss.adc_to_pes, ss.common_threshold, Cal=0)

222 ms ± 3.97 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [64]:
%timeit cpf._signal_sipm(ss.signal_adc, ss.adc_to_pes, ss.common_threshold, Cal=1)

203 ms ± 3.11 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [65]:
%timeit cpf._signal_sipm(ss.signal_adc, ss.adc_to_pes, ss.common_threshold, Cal=2)

200 ms ± 5.06 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
